#### Usage to plot similarity matrix

In [ ]:
#for relative open masslib
import sys
sys.path.append('..')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from masslib.mass import MassSpectrum
from masslib.mass import VanKrevelen
from masslib.mass import Tmds
from masslib.mass import MassSpectrumList
from masslib.brutto_generator import brutto_gen

#### Load spectra

In [ ]:
masses = []
names = []
mapper = {"mw": "mass", "relativeAbundance": "intensity", "formula":"brutto"}
for filename in sorted(os.listdir("data/similarity/")):
    if filename[-3:] != 'csv':
        continue
    names.append(filename[:-4])
    masses.append(MassSpectrum().load(
        f"data/similarity/{filename}",
        mapper=mapper,
        sep=',',
        ignore_columns=["peakNo", "errorPPM", "DBE", "class", "z"]
    ).calculate_mass())  # It's important, that similarity can be calculated only by calculated mass

#### Create MassSpectrumList instance

In [ ]:
collection = MassSpectrumList(masses, names)

#### Calc and Plot similarity matrix

In [ ]:
collection.calculate_similarity()


In [ ]:
collection.draw_similarity()

#### Batch processing


In [ ]:
squares = pd.DataFrame()

for i, spec in enumerate(collection.spectra):

    spec = spec.recallibrate(how='mdm')
    spec = spec.assign(sign=None)
    try:
        spec = spec.assign_by_tmds(max_num=100)
    except:
        pass

    spec.draw()
    plt.title(collection.names[i])
    
    collection.spectra[i] = spec

    vk = VanKrevelen(spec)
    vk.draw_scatter()
    plt.title(f'{collection.names[i]}, {len(spec.drop_unassigned())} formulas')

    square = vk.squares()
    squares[collection.names[i]] = square['value']

squares.index = [i for i in range(1,21)]
print(squares)

In [ ]:
collection.draw_similarity()